### CS4423 - Networks
Prof. Götz Pfeiffer<br />
School of Mathematics, Statistics and Applied Mathematics<br />
NUI Galway

# Lecture 8: Graph Isomorphism and Symmetries

Symmetries, or the lack thereof, are interesting properties of networks ...

According to Cayley's formula, there are $4^{4-2} = 16$
trees on $n = 4$ vertices.  But overall, we only see $2$
distinct structures, a path of length $3$, and a star graph
with $3$ spikes.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
opts = { "with_labels": True, "node_color": 'y'}

As a random graph, the path graph occurs far more often than
the star graph.  Is something wrong with the assumption of uniform
distribution?

In [ ]:
T = nx.random_tree(4)
nx.draw(T)

## Graph Isomorphism

If we are mainly interested in the network **structure**
of a graph $G = (X, E)$, the underlying vertex set $X$ is replacable.
The following definition makes this notion precise.

<div class="alert alert-danger">
<b>Definition.</b>  
    <ul>
        <li> 
            Let $G_1 = (X_1, E_2)$ 
            and $G_2 = (X_2, E_2)$ be graphs.
        </li>
        <li>
            A **graph isomorphism** from $G_1$ to $G_2$
            is a **bijective** map $f \colon X_1 \to X_2$
            such that $(f(x), f(y))$ is an edge of $G_2$
            if and only if $(x, y) \in E_1$.
        </li>
        <li>
            We say that $G_1$ is isomorphic to $G_2$
            (and write $G_1 \equiv G_2$) if
            there is a graph isomorphism $f$ from $G_1$ to $G_2$.
        </li>
        <li>
            In the special case where both $X_1$ and $X_2$
            are the same set $X$,
            a bijection $f \colon X \to X$ is called
            a **permutation** of $X$.
        </li>
        <li>
            If a permutation $f$ of $X$ is a graph isomorphism
            from $G_1 = (X, E_1)$ to $G_2 = (X, E_2)$
            then in fact $E_1 = E_2 =: E$
            and $f$ is called a **graph automorphism** of the
            graph $G:= G_1 = G_2$.
        </li>
    </ul>
</div>

* **Examples.** ...

*  **Note:** the automorphisms of a graph form a **group**.

Any bijection $f \colon X \to Y$ can be used to produce an **isomorphic copy** of a graph $G = (X, E)$

In [ ]:
G = nx.Graph([(1,2)])
H = nx.Graph([(2,1)])

In [ ]:
print(list(G.edges()), list(H.edges()))

In [ ]:
def normalized_edges(G):
    return tuple(sorted(tuple(set(e)) for e in G.edges()))

In [ ]:
def are_equal_as_graphs(G, H):
    if set(G.nodes()) != set(H.nodes()):
        return false
    return normalized_edges(G) == normalized_edges(H)

In [ ]:
are_equal_as_graphs(G, H)

In [ ]:
def list_contains_graph(graphs, G):
    for H in graphs:
        if are_equal_as_graphs(G, H):
            return True
    return False

In [ ]:
list_contains_graph([G], H)

In [ ]:
n = 4
n**(n-2)

In [ ]:
T = nx.random_tree(n)
nx.draw(T, **opts)

* Permutations act on trees by relabelling the nodes
* The **symmetric group** on $X = \{0, 1, 2, \dots, n-1\}$
  is the set of all permutations of $X$.
* It is generated by the **transpositions** of 
  consecutive numbers: $k \leftrightarrow k+1$
* This defines a **graph** on the set of all trees on $X$
* The connected component (unlabelled graph) of `T` can
  be constructed by BFS: the **orbit** of `T` under the
  symmetric group.
* As a by-product, the **automorphism group** of is determined.

## Permutations are Dictionaries

* `python` dictionaries are maps $f \colon \mathrm{keys} \to \mathrm{values}$.
* A permutation of $X$ is a map $f \colon X \to X$.
* $\leadsto$ represent a permutation as a `dict`. 

* The **identity** permutation $f(x) = x$:

In [ ]:
one = { k: k for k in range(n) }
one

* The **transposition** $i \leftrightarrow i{+}1$:

In [ ]:
def transposition(n, i):
    t = { k: k for k in range(n) }
    t[i], t[i+1] = t[i+1], t[i]
    return t

In [ ]:
t1 = transposition(n, 1)
t2 = transposition(n, 2)
print(t1, t2)

* **Composition** of `dict`s:  $(g \circ f)(x) = g(f(x))$.

In [ ]:
def composition(a, b):
    return { k: a[b[k]] for k in b.keys() }

* $(1,2) \circ (2,3) = (1, 2, 3)$,
* $(2,3) \circ (1,2) = (1, 3, 2)$.

In [ ]:
t12 = composition(t1, t2)
t12

In [ ]:
t21 = composition(t2, t1)
t21

* $f^{-1}(y) = x \iff f(x) = y$.

In [ ]:
def inverse(a):
    return { v : k for k, v in a.items() }

* $(1,2,3)^{-1} = (1,3,2)$.

In [ ]:
inverse(t21)

In [ ]:
inverse(t21)  == t12

* The **symmetric group** $S_n$ of all permutations of $X = \{0, 1, \dots, n-1\}$ is generated by the transpositions $(i, i{+}1)$, $i = 0, \dots, n-2$.

In [ ]:
gens = [transposition(n, k) for k in range(n-1)]

* BFS systematically computes all products of the generators:

In [ ]:
def elements(gens, one):
    orbit = [one]
    for x in orbit:
        for a in gens:
            y = composition(a, x)
            if not y in orbit:
                orbit.append(y)
    return orbit

In [ ]:
sym = elements(gens, one)
len(sym)

## Orbits of Isomorphic Trees

* The process of relabelling the nodes of a graph with vertex set
  $X$ by permutations of $X$ defines a **group action** of $S_n$
  on the set of all labelled trees on $X$.
  
* $1_X(T) = T$ and $(g \circ f)(T) = g(f(T))$ for each tree $T$
  with vertex set $X$.

In [ ]:
G = nx.path_graph(6)
G12 = nx.relabel_nodes(nx.relabel_nodes(G, t1), t2)
print(list(G12.nodes()))
GG = nx.relabel_nodes(G, t21)
print(list(GG.nodes()))

*  The **orbit** of $T$ (under $S_n$) is the set of all images
   $f(T)$, $f \in S_n$.
   
* **BFS** sytematically computes the orbit by repeatedly applying
  the generators of $S_n$:

In [ ]:
orbit = [T]
for x in orbit:
    for a in gens:
        y = nx.relabel_nodes(x, a)
        if not list_contains_graph(orbit, y):
            orbit.append(y)

In [ ]:
len(orbit)

* Graph and Queue version:

In [ ]:
G = nx.Graph()
G.add_node(T)
Q = [T]

In [ ]:
while len(Q) > 0:
    x = Q.pop(0)
    for a in gens:
        y = nx.relabel_nodes(x, a)
        y = next((z for z in G if are_equal_as_graphs(y, z)), y)
        if not y in G:
            G.add_node(y)
            Q.append(y)
            G.add_edge(x, y)
        else:
#            if y != x:
                G.add_edge(x, y)      

In [ ]:
nx.draw(G)

* orbit + schreier transversal.


In [ ]:
T.graph['perm'] = { i: i for i in range(n) }
orbit = [T]
for x in orbit:
    for a in gens:
        y = nx.relabel_nodes(x, a)
        if not list_contains_graph(orbit, y):
            y.graph['perm'] = composition(a, x.graph['perm'])
            orbit.append(y)
        

In [ ]:
[x.graph['perm'] for x in orbit]

* Orbit and Schreier generators
* Replace `y` by its copy `z` in `orbit` if there is one.
* if not, set `y.graph['perm']` and append `y` to `orbit`.

In [ ]:
autos = []
T.graph['perm'] = { i: i for i in range(n) }
orbit = [T]
for x in orbit:
    for a in gens:
        y = nx.relabel_nodes(x, a)
        ax = composition(a, x.graph['perm'])
        z = next((z for z in orbit if are_equal_as_graphs(y, z)), None)
        if z:
            z1ax = composition(inverse(z.graph['perm']), ax)
            if not z1ax in autos:
                autos.append(z1ax)
        else:
            y.graph['perm'] = ax
            orbit.append(y)        

In [ ]:
[list(a.values()) for a in autos]

In [ ]:
len(elements(autos, one))

* `python` function version

In [ ]:
def automorphisms(tree):
    
    # initialize
    n = tree.order()
    gens = [transposition(n, i) for i in range(n-1)]
    autos = []
    tree.graph['perm'] = { i: i for i in range(n) }
    orbit = [tree]
    
    # loop
    for x in orbit:
        for a in gens:
            ax = composition(a, x.graph['perm'])
            y = nx.relabel_nodes(x, a)
            z = next((z for z in orbit if are_equal_as_graphs(y, z)), None)
            if z:
                z1ax = composition(inverse(z.graph['perm']), ax)
                if not z1ax in autos:
                    autos.append(z1ax)
            else:
                y.graph['perm'] = ax
                orbit.append(y)

    # return result
    return autos

In [ ]:
T = nx.random_tree(6)
nx.draw(T, **opts)

In [ ]:
autos = automorphisms(T)
print([list(a.values()) for a in autos])
one = { k: k for k in range(T.order()) }
len(elements(autos, one))

##  Code Corner

### `python`

* `sorted` [doc]
* `tuple` [doc]
* `set` [doc]
* `next` [doc]
* `[].pop` [doc]
* `{}.keys` [doc]
* `{}.values` [doc]
* `{}.items` [doc]

### `networkx`

* `relabel_nodes`
* `path_graph`

## Exercises

1.  How many unlabelled trees are there on $n = 5$ vertices?
   What (sizes) are their automorphism groups?

2. $n = 6$?